In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# import libraries 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from PIL import Image
from glob import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf

from  tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision,Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.models import Model

In [ ]:
print(tf.__version__)
print()

# preprocessing 

In [ ]:
def train_df(tr_path):
    classes, class_paths = zip(*[(label, os.path.join(tr_path, label, image))
                                 for label in os.listdir(tr_path) if os.path.isdir(os.path.join(tr_path, label))
                                 for image in os.listdir(os.path.join(tr_path, label))])

    tr_df = pd.DataFrame({'Class Path': class_paths, 'Class': classes})
    return tr_df       

In [ ]:
def test_df(ts_path):
    classes, class_paths = zip(*[(label, os.path.join(ts_path, label, image))
                                 for label in os.listdir(ts_path) if os.path.isdir(os.path.join(ts_path, label))
                                 for image in os.listdir(os.path.join(ts_path, label))])

    ts_df = pd.DataFrame({'Class Path': class_paths, 'Class': classes})
    return ts_df

In [ ]:
tr_df =train_df('/kaggle/input/brain-tumor-mri-dataset/Training')



In [ ]:
tr_df

In [ ]:
ts_df =test_df('/kaggle/input/brain-tumor-mri-dataset/Testing')


In [ ]:
ts_df

In [ ]:
plt.figure(figsize=(15,7))
ax =sns.countplot(data =tr_df,y =tr_df['Class'])

plt.xlabel(' ')
plt.ylabel(' ')
plt.title('count of images in each class',fontsize=20)
ax.bar_label(ax.containers[0])
plt.show()



In [ ]:
#Count each class in test data
plt.figure(figsize=(15, 7))
ax = sns.countplot(y=ts_df['Class'], palette='viridis')

ax.set(xlabel='', ylabel='', title='Count of images in each class')
ax.bar_label(ax.containers[0])

plt.show()

# *split data INTO TRAIN ,TEST,VALID*

In [ ]:
valid_df,ts_df =train_test_split(ts_df,train_size=0.5,random_state=20,stratify=ts_df['Class'])


In [ ]:
valid_df

In [ ]:
ts_df

# Data preprocessing

In [ ]:
batch_size = 32
img_size = (299, 299)

_gen = ImageDataGenerator(rescale=1/255,
                          brightness_range=(0.8, 1.2))

ts_gen = ImageDataGenerator(rescale=1/255)


tr_gen = _gen.flow_from_dataframe(tr_df, x_col='Class Path',
                                  y_col='Class', batch_size=batch_size,
                                  target_size=img_size)

valid_gen = _gen.flow_from_dataframe(valid_df, x_col='Class Path',
                                     y_col='Class', batch_size=batch_size,
                                     target_size=img_size)

ts_gen = ts_gen.flow_from_dataframe(ts_df, x_col='Class Path',
                                  y_col='Class', batch_size=16,
                                  target_size=img_size, shuffle=False)

# GETTING SAMPLE DATA

In [ ]:
class_dict = tr_gen.class_indices
classes = list(class_dict.keys())
images, labels = next(ts_gen)

plt.figure(figsize=(20, 20))

for i, (image, label) in enumerate(zip(images, labels)):
    plt.subplot(4,4, i + 1)
    plt.imshow(image)
    class_name = classes[np.argmax(label)]
    plt.title(class_name, color='k', fontsize=15)

plt.show()

In [ ]:
img_shape =(229,229,3)
base_model =tf.keras.applications.Xception(include_top =False,weights='imagenet',input_shape=img_shape,pooling='max')


In [ ]:
from tensorflow.keras.optimizers import Adamax

# building model

In [ ]:
img_shape=(299,299,3)
base_model = tf.keras.applications.Xception(include_top= False, weights= "imagenet",
                            input_shape= img_shape, pooling= 'max')

# for layer in base_model.layers:
#     layer.trainable = False
    
model = Sequential([
    base_model,
    Flatten(),
    Dropout(rate= 0.3),
    Dense(128, activation= 'relu'),
    Dropout(rate= 0.25),
    Dense(4, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001),
              loss= 'categorical_crossentropy',
              metrics= ['accuracy',
                        Precision(),
                        Recall()])

model.summary()

In [ ]:
hist = model.fit(tr_gen,
                 epochs=10,
                 validation_data=valid_gen,
                 shuffle= False)

In [ ]:
hist.history.keys()


# visulalizing model performance

In [ ]:
tr_acc =hist.history['accuracy']
tr_loss =hist.history['loss']
tr_per =hist.history['precision_5']
tr_recall =hist.history['recall_5']

val_acc =hist.history['val_accuracy']
val_loss =hist.history['val_loss']
val_per =hist.history['val_precision_5']
val_recall =hist.history['val_recall_5']

plt.figure(figsize=(20, 12))
plt.style.use('fivethirtyeight')

plt.subplot(2,2,1)
plt.plot(tr_loss,'r',label='train loss')
plt.plot(val_loss,'g',label='validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(2,2,2)
plt.plot(tr_acc,'r',label='training accuracy')
plt.plot(val_acc,'g',label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.grid(True)


plt.subplot(2,2,3)
plt.plot(tr_recall,'r',label='recall_5')
plt.plot(val_recall,'g',label='validation recall_5')
plt.title('Recall and Validation Recall')
plt.xlabel('Epochs')
plt.ylabel('Recall_5')
plt.legend()
plt.grid(True)

plt.suptitle('Model Training Metrics Over Epochs', fontsize=16)
plt.show()

# test and evaluation

In [ ]:
train_score =model.evaluate(tr_gen,verbose=1)
valid_score =model.evaluate(valid_gen,verbose=1)
test_score =model.evaluate(ts_gen,verbose=1)

print(f"train loss:{train_score[0]:.4f}")
print(f"train accuracy:{train_score[1]*100:2f}%")
print('_'*20)
print(f"validation loss:{valid_score[0]:.4f}")
print(f"val accuracy:{valid_score[1]*100:2f}%")
print('_'*20)
print(f"Test Loss: {test_score[0]:.4f}")
print(f"Test Accuracy: {test_score[1]*100:.2f}%")

In [ ]:
preds =model.predict(ts_gen)
y_pred =np.argmax(preds,axis=1)

In [ ]:
cm =confusion_matrix(ts_gen.classes,y_pred)
labels =list(class_dict.keys())
plt.figure(figsize =(10,8))
sns.heatmap(cm,annot=True,fmt='d',cmap='Blues',
           xticklabels=labels,yticklabels=labels)
plt.xlabel('predicted label')
plt.ylabel('truth label')
plt.show()

In [ ]:
clr =classification_report(ts_gen.classes,y_pred)
print(clr)

# *save the model*

In [ ]:
model.save("Xception_model.keras")

# predict the images

In [ ]:
def predict(img_path):
    import numpy as np
    import matplotlib.pyplot as plt
    from PIL import Image
    label =list(class_dict.keys())
    plt.figure(figsize=(12,12))
    img =Image.open(img_path)
    resized_img =img.resize((299,299))
    img =np.asarray(resized_img)
    img =np.expand_dims(img,axis=0)
    img =img/255
    predictions =model.predict(img)
    probs =list(predictions[0])
    labels =label
    plt.subplot(2,2,1)
    plt.imshow(resized_img)
    plt.subplot(2,1,2)
    bars  =plt.barh(labels,probs)
    plt.xlabel('Probalility',fontsize=15)
    ax =plt.gca()
    ax.bar_label(bars,fmt='%.f')
    plt.show()

In [ ]:
predict('/kaggle/input/brain-tumor-mri-dataset/Testing/meningioma/Te-meTr_0001.jpg')


In [ ]:
predict('/kaggle/input/brain-tumor-mri-dataset/Testing/glioma/Te-glTr_0002.jpg')

In [ ]:
predict('/kaggle/input/brain-tumor-mri-dataset/Testing/pituitary/Te-piTr_0001.jpg')

# 